In [103]:
import java.nio.file.Files
import java.nio.file.Path

data class BenchmarkData(val name: String, val score: Double, val error: Double) {
    override fun toString(): String = "$name: $score ± $error ns/op"
}

val benchmarkLines = Files.readAllLines(Path.of("build", "results", "jmh", "results.txt"))
    .mapNotNull {
        val r = Regex("JmhBenchmark\\.([^ ]+) +avgt +\\d+ +([0-9.]+) +± +([0-9.]+) +ns/op")
        val match = r.matchEntire(it) ?: return@mapNotNull null
        BenchmarkData(
            match.groups[1]!!.value,
            match.groups[2]!!.value.toDouble(),
            match.groups[3]!!.value.toDouble(),
        )
    }
benchmarkLines.joinToString("\n")

baselineDouble: 936774.42 ± 1380.126 ns/op
baselineFloat: 923287.075 ± 11462.162 ns/op
inlineDouble: 932035.84 ± 4943.213 ns/op
inlineFloat: 897810.246 ± 1695.109 ns/op
longPackFloat: 1585716.339 ± 25744.925 ns/op
mutableRefDouble: 1093688.2 ± 1988.159 ns/op
mutableRefFloatIn1Long: 1748488.235 ± 5183.316 ns/op
mutableRefFloatIn2Longs: 1079365.369 ± 3498.671 ns/op
valueDouble: 655928.157 ± 2584.086 ns/op
valueFloat: 668845.568 ± 18528.289 ns/op
valueInlineDouble: 651329.162 ± 6638.435 ns/op
valueInlineFloat: 685287.53 ± 1895.324 ns/op

In [104]:
%use lets-plot

In [105]:
import org.jetbrains.letsPlot.intern.PosKind
import org.jetbrains.letsPlot.intern.StatKind
import org.jetbrains.letsPlot.intern.layer.PosOptions
import org.jetbrains.letsPlot.intern.layer.StatOptions

val names = benchmarkLines.map { it.name }
val scores = benchmarkLines.map { it.score }
val scoresMinusErrors = benchmarkLines.map { it.score - it.error / 2 }
val scoresPlusErrors = benchmarkLines.map { it.score + it.error / 2 }
val baseline = listOf("Float", "Double").associateWith { type ->
    benchmarkLines.single { it.name == "baseline$type" }
}
val plot = letsPlot() +
        labs("Microbenchmark of 3D scene rendering", fill = "Benchmark", y = "ns/op") +
        geomBar(stat = StatOptions(kind = StatKind.IDENTITY)) {
            x = names
            y = scores
            fill = names
        } +
        geomErrorBar(
            width = 0.5,
            position = PosOptions(kind = PosKind.DODGE),
            size = 0.6
        ) {
            x = names
            ymin = scoresMinusErrors
            ymax = scoresPlusErrors
        } +
        geomText(
            stat = StatOptions(kind = StatKind.IDENTITY),
            position = positionStack(vjust = 0.5),
            size = 5,
        ) {
            x = names
            y = scores
            label = benchmarkLines.map {
                val isFloat = it.name.contains("Float")
                val type = if (isFloat) "Float" else "Double"
                val score = it.score / baseline[type]!!.score
                val error = it.error / baseline[type]!!.score
                "${String.format("%.0f", score * 100)}%\n±\n${String.format("%.1f", error * 100)}%\n${type}"
            }
        } +
        scaleXDiscrete(
            labels = names,
            name = "",
            limits = names + listOf(""),
            breaks = names.toList()
        ) +
        scaleYContinuous(limits = 0 to scoresPlusErrors.max(), name = "ns/op") +
        theme(plotTitle = elementText(hjust = 0.5)).legendPositionRight() +
        ggsize(1000, 800)


plot.show()

In [106]:
ggsave(plot = plot, filename = "results.svg")

/Users/Evgeniy.Zhelenskiy/IdeaProjects/Wolf3d/lets-plot-images/results.svg